# Régression logistique
Dans ce cahier, nous implémenterons la régression logistique pour l'analyse des sentiments sur les tweets. Étant donné un tweet, vous déciderez s’il a un sentiment positif ou négatif. Plus précisément, vous allez :

* Apprenez à extraire des fonctionnalités pour la régression logistique à partir de texte
* Implémenter la régression logistique à partir de zéro
* Appliquer la régression logistique sur une tâche de traitement du langage naturel
* Testez en utilisant votre régression logistique
* Effectuer une analyse des erreurs

## Importer des fonctions et des données

In [25]:
import re
import nltk
import string
import numpy as np
import pandas as pd



from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import twitter_samples



nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
def process_tweet(tweet):
    """Traitement de la fonction tweet.
     Saisir:
         tweet : une chaîne contenant un tweet
     Sortir:
         tweets_clean : une liste de mots contenant le tweet traité
     """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # supprimer les tickers boursiers comme $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # supprimer l'ancien texte de retweet "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # supprimer les hyperliens
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # supprimer les hashtags
    # en supprimant uniquement le signe dièse # du mot
    tweet = re.sub(r'#', '', tweet)
    # tokeniser les tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # supprimer les mots vides
                word not in string.punctuation):  # supprimer la ponctuation

            stem_word = stemmer.stem(word)  # mot dérivé
            tweets_clean.append(stem_word)

    return tweets_clean


def build_freqs(tweets, ys):
    """Construisez des fréquences.
     Saisir:
         tweets : une liste de tweets
         ys : un tableau m x 1 avec l'étiquette de sentiment de chaque tweet
             (soit 0 ou 1)
     Sortir:
         freqs : un dictionnaire mappant chaque paire (mot, sentiment) à son
         fréquence
     """
    # Convertissez le tableau np en liste car zip a besoin d'un itérable.
    # Le squeeze est nécessaire ou la liste se termine avec un élément.
    # Notez également qu'il ne s'agit que d'un NOP si ys est déjà une liste.
    yslist = np.squeeze(ys).tolist()

    # Commencez avec un dictionnaire vide et remplissez-le en parcourant tous les tweets
    # et sur tous les mots traités dans chaque tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

### Préparer les données
* Le `twitter_samples` contient des sous-ensembles de cinq mille tweets_positifs, cinq mille tweets_négatifs et l'ensemble complet de 10 000 tweets.
     * Si vous utilisiez les trois ensembles de données, nous introduirions des doublons des tweets positifs et des tweets négatifs.
     * Vous sélectionnerez uniquement les cinq mille tweets positifs et les cinq mille tweets négatifs.

In [27]:
# sélectionnez l'ensemble des tweets positifs et négatifs
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Répartition des tests d'entraînement : 20 % seront dans l'ensemble de tests et 80 % dans l'ensemble d'entraînement.

In [28]:
# diviser les données en deux parties, une pour la formation et une pour les tests (ensemble de validation)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

* Créez le tableau numpy d'étiquettes positives et d'étiquettes négatives.

In [29]:
# combiner des étiquettes positives et négatives
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [30]:
# Imprimer le train de formes et les ensembles de tests
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


* Créez le dictionnaire de fréquences à l'aide de la fonction build_freqs importée.
     * Nous vous recommandons fortement d'ouvrir utils.py et de lire la fonction build_freqs pour comprendre ce qu'elle fait.
     * Pour afficher le répertoire de fichiers, allez dans le menu et cliquez sur Fichier->Ouvrir.

```Python
    for y,tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
```
* Remarquez comment la boucle for externe parcourt chaque tweet et la boucle for interne parcourt chaque mot d'un tweet.
* Le dictionnaire 'freqs' est le dictionnaire de fréquences en cours de construction.
* La clé est le tuple (mot, étiquette), tel que ("happy",1) ou ("happy",0). La valeur stockée pour chaque clé correspond au nombre de fois où le mot « heureux » a été associé à une étiquette positive, ou combien de fois « heureux » a été associé à une étiquette négative.

In [31]:
# créer un dictionnaire de fréquences
freqs = build_freqs(train_x, train_y)

# vérifie le résultat
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11337


### Traiter le tweet
La fonction donnée 'process_tweet' tokenise le tweet en mots individuels, supprime les mots vides et applique la radicalisation.

In [32]:
# testez la fonction ci-dessous
print('Ceci est un exemple de tweet positif: \n', train_x[0])
print('\nCeci est un exemple de la version traitée du tweet: \n', process_tweet(train_x[0]))

Ceci est un exemple de tweet positif: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

Ceci est un exemple de la version traitée du tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']



## 1 – Régression logistique

### 1.1 - Sigmoïde
Vous apprendrez à utiliser la régression logistique pour la classification de textes.
* La fonction sigmoïde est définie comme :

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

Il mappe l'entrée « z » à une valeur comprise entre 0 et 1 et peut donc être traitée comme une probabilité.


In [33]:

def sigmoid(z):
    '''
     Saisir:
         z : est l'entrée (peut être un scalaire ou un tableau)
     Sortir:
         h : le sigmoïde de z
     '''
    # calculer la sigmoïde de z
    h = 1/(1+np.exp(-z))
    return h

#### Régression logistique : régression et sigmoïde

La régression logistique utilise une régression linéaire régulière et applique une sigmoïde au résultat de la régression linéaire.

Régression:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Notez que les valeurs $\theta$ sont des « poids ». Si vous avez suivi la spécialisation deep learning, nous avons fait référence aux poids avec le vecteur « w ». Dans ce cours, nous utilisons une variable différente $\theta$ pour faire référence aux poids.

Régression logistique
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
We will refer to 'z' as the 'logits'.


### 1.2 - Fonction de coût et gradient

La fonction de coût utilisée pour la régression logistique est la moyenne de la perte logarithmique pour tous les exemples de formation :

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ est le nombre d'exemples de formation
* $y^{(i)}$ est l'étiquette réelle de l'exemple de formation « i ».
* $h(z^{(i)})$ est la prédiction du modèle pour l'exemple d'entraînement « i ».

La fonction de perte pour un seul exemple de formation est
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* Toutes les valeurs $h$ sont comprises entre 0 et 1, les logs seront donc négatifs. C'est la raison pour laquelle le facteur -1 est appliqué à la somme des deux termes de perte.
* Notez que lorsque le modèle prédit 1 ($h(z(\theta)) = 1$) et que l'étiquette « y » est également 1, la perte pour cet exemple de formation est de 0.
* De même, lorsque le modèle prédit 0 ($h(z(\theta)) = 0$) et que l'étiquette réelle est également 0, la perte pour cet exemple de formation est de 0.
* Cependant, lorsque la prédiction du modèle est proche de 1 ($h(z(\theta)) = 0,9999$) et que l'étiquette est 0, le deuxième terme de la perte log devient un grand nombre négatif, qui est ensuite multiplié par le facteur global de -1 pour le convertir en une valeur de perte positive. $-1 \times (1 - 0) \times log(1 - 0,9999) \approx 9,2$ Plus la prédiction du modèle se rapproche de 1, plus la perte est importante.

* De même, si le modèle prédit proche de 0 ($h(z) = 0,0001$) mais que l'étiquette réelle est 1, le premier terme de la fonction de perte devient un grand nombre : $-1 \times log(0,0001) \approx 9,2$. Plus la prédiction est proche de zéro, plus la perte est importante.

In [34]:
# vérifier que lorsque le modèle prédit proche de 1, mais que l'étiquette réelle est 0, la perte est une grande valeur positive
-1 * (1 - 0) * np.log(1 - 0.9999) # loss is about 9.2

9.210340371976294

In [35]:
# vérifier que lorsque le modèle prédit proche de 0 mais que l'étiquette réelle est 1, la perte est une grande valeur positive
-1 * np.log(0.0001) # loss is about 9.2

9.210340371976182

#### Mettre à jour les poids

Pour mettre à jour votre vecteur de poids $\theta$, vous appliquerez une descente de gradient pour améliorer de manière itérative les prédictions de votre modèle.
Le gradient de la fonction de coût $J$ par rapport à l'un des poids $\theta_j$ est :

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x^{(i)}_j \tag{5}$$
* « i » est l'index de tous les exemples de formation « m ».
* 'j' est l'indice du poids $\theta_j$, donc $x^{(i)}_j$ est la caractéristique associée au poids $\theta_j$

* Pour mettre à jour le poids $\theta_j$, on l'ajuste en soustrayant une fraction du gradient déterminé par $\alpha$ :
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* Le taux d'apprentissage $\alpha$ est une valeur que nous choisissons pour contrôler la taille d'une seule mise à jour.


### Exercice 2 - gradientDescent
Implémentez la fonction de descente de gradient.
* Le nombre d'itérations « num_iters » est le nombre de fois que vous utiliserez l'intégralité de l'ensemble d'entraînement.
* Pour chaque itération, vous calculerez la fonction de coût en utilisant tous les exemples de formation (il existe « m » exemples de formation) et pour toutes les fonctionnalités.
* Au lieu de mettre à jour un seul poids $\theta_i$ à la fois, nous pouvons mettre à jour tous les poids du vecteur colonne :  
$$\mathbf{\theta} = \begin{pmatrix}
\theta_0
\\
\theta_1
\\
\theta_2
\\
\vdots
\\
\theta_n
\end{pmatrix}$$
* $\mathbf{\theta}$ a des dimensions (n+1, 1), où 'n' est le nombre de fonctionnalités, et il y a un élément supplémentaire pour le terme de biais $\theta_0$ (notez que la valeur de fonctionnalité correspondante $\mathbf{x_0}$ est 1).
* Les « logits », « z », sont calculés en multipliant la matrice de caractéristiques « x » par le vecteur de poids « thêta ».  $z = \mathbf{x}\mathbf{\theta}$
    * $\mathbf{x}$ has dimensions (m, n+1)
    * $\mathbf{\theta}$: has dimensions (n+1, 1)
    * $\mathbf{z}$: has dimensions (m, 1)
* La prédiction 'h', est calculée en appliquant la sigmoïde à chaque élément de 'z' : $h(z) = sigmoïde(z)$, et a des dimensions (m,1).
* La fonction de coût $J$ est calculée en prenant le produit scalaire des vecteurs 'y' et 'log(h)'. Puisque « y » et « h » sont tous deux des vecteurs colonnes (m,1), transposez le vecteur vers la gauche, de sorte que la multiplication matricielle d'un vecteur ligne avec un vecteur colonne effectue le produit scalaire.
$$J = \frac{-1}{m} \times \left(\mathbf{y}^T \cdot log(\mathbf{h}) + \mathbf{(1-y)}^T \cdot log(\mathbf{1-h}) \right)$$
* La mise à jour de theta est également vectorisée. Parce que les dimensions de $\mathbf{x}$ sont (m, n+1), et que $\mathbf{h}$ et $\mathbf{y}$ sont (m, 1), nous devons transposer le $ \mathbf{x}$ et placez-le à gauche afin d'effectuer une multiplication matricielle, qui donne alors la réponse (n+1, 1) dont nous avons besoin :
$$\mathbf{\theta} = \mathbf{\theta} - \frac{\alpha}{m} \times \left( \mathbf{x}^T \cdot \left( \mathbf{h-y} \right) \right)$$

In [36]:

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
     Saisir:
         x : matrice de caractéristiques qui est (m,n+1)
         y : étiquettes correspondantes de la matrice d'entrée x, dimensions (m,1)
         thêta : vecteur poids de dimension (n+1,1)
         alpha : taux d'apprentissage
         num_iters : nombre d'itérations pour lesquelles vous souhaitez entraîner votre modèle
     Sortir:
         J : le coût final
         thêta : votre vecteur de poids final
     Astuce : vous souhaiterez peut-être imprimer le coût pour vous assurer qu'il diminue.
     '''

    # récupère 'm', le nombre de lignes dans la matrice x
    m = x.shape[0]

    for i in range(0, num_iters):

        # obtenir z, le produit scalaire de x et thêta
        z = np.dot(x, theta)

        # obtenir le sigmoïde de z
        h = sigmoid(z)

        # calculer la fonction de coût
        J = (-1/m)*(np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))

        # mettre à jour les poids thêta
        theta = theta - (alpha/m)*np.dot(x.T, (h-y))

    J = float(J)
    return J, theta

In [37]:
# Vérifiez la fonction
# Construire un scénario de test synthétique en utilisant les fonctions numpy PRNG
np.random.seed(1)
# L'entrée X est 10 x 3 avec des uns pour les termes de biais
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Les étiquettes Y sont 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Appliquer une descente de dégradé
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


<a name='2'></a>
## 2 - Extraire les fonctionnalités

* Étant donné une liste de tweets, extrayez les fonctionnalités et stockez-les dans une matrice. Vous allez extraire deux fonctionnalités.
     * La première caractéristique est le nombre de mots positifs dans un tweet.
     * La deuxième caractéristique est le nombre de mots négatifs dans un tweet.
* Entraînez ensuite votre classificateur de régression logistique sur ces fonctionnalités.
* Testez le classificateur sur un ensemble de validation.


### Exercice 3 - extract_features
Implémentez la fonction extract_features.
* Cette fonction prend en compte un seul tweet.
* Traitez le tweet à l'aide de la fonction importée `process_tweet` et enregistrez la liste des mots du tweet.
* Parcourez chaque mot de la liste des mots traités
     * Pour chaque mot, vérifiez le dictionnaire « freqs » pour connaître le nombre de mots lorsque ce mot a une étiquette « 1 » positive. (Vérifiez la clé (mot, 1.0)
     * Faites de même pour le décompte lorsque le mot est associé à l'étiquette négative « 0 ». (Vérifiez la clé (mot, 0.0).)

**Remarque :** Dans les instructions d'implémentation fournies ci-dessus, la prédiction d'être positif ou négatif dépend du vecteur de caractéristiques qui compte les mots en double - ceci est différent de ce que vous avez vu dans les vidéos de conférence.

In [38]:

def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
     Saisir:
         tweet : une liste de mots pour un tweet
         freqs : un dictionnaire correspondant aux fréquences de chaque tuple (mot, label)
     Sortir:
         x : un vecteur caractéristique de dimension (1,3)
     '''
    # process_tweet tokenise, racine et supprime les mots vides
    word_l = process_tweet(tweet)

    # 3 éléments pour les comptes [biais, positifs, négatifs]
    x = np.zeros(3)

    # terme de biais est défini sur 1
    x[0] = 1



    # boucle sur chaque mot de la liste de mots
    for word in word_l:

        # incrémente le nombre de mots pour l'étiquette positive 1
        if (word, 1) in freqs:
            x[1] += freqs[(word, 1)]

        # incrémente le nombre de mots pour l'étiquette négative 0
        if (word, 0) in freqs:
            x[2] += freqs[(word, 0)]



    x = x[None, :]  # ajout d'une dimension de lot pour un traitement ultérieur
    assert(x.shape == (1, 3))
    return x

In [39]:

tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [40]:

# vérifie quand les mots ne sont pas dans le dictionnaire de fréquences
tmp2 = extract_features('blorb bleeeeb bloooob', freqs)
print(tmp2)

[[1. 0. 0.]]



## 3 – Entraîner votre modèle

Pour entraîner le modèle :
* Empilez les fonctionnalités de tous les exemples de formation dans une matrice X.
* Appelez `gradientDescent`, que vous avez implémenté ci-dessus.

Cette rubrique vous est donnée. Veuillez le lire pour comprendre et exécuter la cellule.

In [41]:
# collecter les caractéristiques 'x' et les empiler dans une matrice 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# libellés de formation correspondant à X
Y = train_y

# Appliquer une descente de dégradé
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

The cost after training is 0.24215478.
The resulting vector of weights is [7e-08, 0.00052391, -0.00055517]



## 4 – Testez votre régression logistique

Il est temps pour vous de tester votre fonction de régression logistique sur de nouvelles entrées que votre modèle n'a jamais vues auparavant.

### Exercice 4 - prédict_tweet
Implémentez `predict_tweet`.
Prédisez si un tweet est positif ou négatif.

* Étant donné un tweet, traitez-le, puis extrayez les fonctionnalités.
* Appliquez les poids appris du modèle sur les fonctionnalités pour obtenir les logits.
* Appliquez le sigmoïde aux logits pour obtenir la prédiction (une valeur comprise entre 0 et 1).

$$y_{pred} = sigmoïde(\mathbf{x} \cdot \theta)$$

In [42]:

def predict_tweet(tweet, freqs, theta):
    '''
     Saisir:
         tweet : une chaîne
         freqs : un dictionnaire correspondant aux fréquences de chaque tuple (mot, label)
         thêta : (3,1) vecteur de poids
     Sortir:
         y_pred : la probabilité qu'un tweet soit positif ou négatif
     '''


    # extraire les fonctionnalités du tweet et le stocker dans x
    x = extract_features(tweet, freqs)

    # faire la prédiction en utilisant x et theta
    z = np.dot(x, theta)
    h = sigmoid(z)
    y_pred = h



    return y_pred

In [43]:
# Exécutez cette cellule pour tester votre fonction
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


I am happy -> 0.518581
I am bad -> 0.494339
this movie should have been great. -> 0.515331
great -> 0.515464
great great -> 0.530899
great great great -> 0.546274
great great great great -> 0.561562


In [44]:
# N'hésitez pas à vérifier le sentiment de votre propre tweet ci-dessous
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, freqs, theta)

array([[0.8163691]])

<a name='4-1'></a>
### 4.1 - Vérifier les performances à l'aide de l'ensemble de test
Après avoir entraîné votre modèle à l'aide de l'ensemble d'entraînement ci-dessus, vérifiez ses performances sur des données réelles et invisibles, en le testant par rapport à l'ensemble de test.

<a name='ex-5'></a>
### Exercice 5 - test_logistic_regression
Implémentez `test_logistic_regression`.
* Compte tenu des données de test et des poids de votre modèle entraîné, calculez la précision de votre modèle de régression logistique.
* Utilisez votre fonction 'predict_tweet' pour faire des prédictions sur chaque tweet de l'ensemble de test.
* Si la prédiction est > 0,5, définissez la classification « y_hat » du modèle sur 1, sinon définissez la classification « y_hat » du modèle sur 0.
* Une prédiction est exacte lorsque le y_hat est égal au test_y. Résumez toutes les instances où elles sont égales et divisez par m.

In [45]:

def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
     Saisir:
         test_x : une liste de tweets
         test_y : (m, 1) vecteur avec les labels correspondants pour la liste des tweets
         freqs : un dictionnaire avec la fréquence de chaque paire (ou tuple)
         thêta : vecteur poids de dimension (3, 1)
     Sortir:
         précision : (nombre de tweets classés correctement) / (nombre total de tweets)
     """


    # la liste de stockage des prédictions
    y_hat = []

    for tweet in test_x:
        # obtenir la prédiction du label pour le tweet
        y_pred = predict_tweet(tweet, freqs, theta)

        if y_pred > 0.5:
            # ajouter 1.0 à la liste
            y_hat.append(1.0)
        else:
            # ajouter 0 à la liste
            y_hat.append(0.0)

    # Avec l'implémentation ci-dessus, y_hat est une liste, mais test_y est un tableau (m,1)
     # convertir les deux en tableaux unidimensionnels afin de les comparer en utilisant l'opérateur '=='
    accuracy = np.sum(np.array(y_hat).flatten() == test_y.flatten())/len(y_hat)



    return accuracy

In [46]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Précision du modèle de régression logistique = {tmp_accuracy:.4f}")

Précision du modèle de régression logistique = 0.9950


<a name='6'></a>
## 6 - Prédisez avec votre propre Tweet

In [48]:
# N'hésitez pas à modifier le tweet ci-dessous
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Sentiment positif')
else:
    print('Sentiment négatif')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
[[0.48139084]]
Sentiment négatif
